# Description:

This is used to analyze the results of training model by cv.



In [3]:
import os
import glob
import re
import numpy as np
import pandas as pd

In [4]:
res_root_dir = '/zfssz2/ST_MCHRI/BIGDATA/PROJECT/NIPT_CNV/1dcnn_resnet'
cv_res_dir = os.path.join(res_root_dir, 'models2')

# this is the results from keras cvslogger callback
cv_metrics_dir = os.path.join(cv_res_dir,'model_metrics')

In [12]:
def get_high_score(fname):
    df_score = pd.read_csv(fname)
    res = df_score.loc[df_score['val_acc'].idxmax()]
    return int(res['epoch']+1), res['acc'], res['loss'], res['val_acc'], res['val_loss']
    

In [13]:
# define the dataframe to save best result
col_name = ['batch_size', 'epoch', 'learn_rate', 
            'learn_rate_decay', 'filters', 'drop', 
            'fc_size', 'blocks', 'kernel_regular',
            'best_epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc']

# we also need to determine 
#     1, the use of fully connection layers 
#        or global_average_pooling layers.
#     2, the use of attention layers or not.

In [14]:
batch_size = []
epoch = []
lr = []
lr_decay = []
filters = []
drop = []
fc_size = []
blocks = []
kernel_reguler = []
best_epoch = []
train_loss = []
train_acc = []
val_loss = []
val_acc = []
re_exp = r'[-+]?\d*\.\d+|\d+|None'
for fname in glob.glob(cv_metrics_dir+'/*'):
    sc = get_high_score(fname)
    hps_str = os.path.basename(fname).split('-')[0]
    hps_lst = re.findall(re_exp, hps_str)
    batch_size.append(int(hps_lst[0]))
    epoch.append(int(hps_lst[1]))
    lr.append(float(hps_lst[2]))
    lr_decay.append(float(hps_lst[3]))
    filters.append(int(hps_lst[4]))
    drop.append(float(hps_lst[5]))
    fc_size.append(int(hps_lst[6]))
    blocks.append(int(hps_lst[7]))
    if len(hps_lst) <= 8 or hps_lst[8] == 'None':
        kernel_reguler.append(np.NAN)
    elif len(hps_lst) > 8:
        kernel_reguler.append(float(hps_lst[8]))
    best_epoch.append(sc[0])
    train_loss.append(sc[1])
    train_acc.append(sc[2])
    val_loss.append(sc[3])
    val_acc.append(sc[4])
df_res = pd.DataFrame(data={'batch_size':batch_size, 'epoch':epoch, 'learn_rate':lr, 
            'learn_rate_decay': lr_decay, 'filters':filters, 'drop':drop, 
            'fc_size':fc_size, 'blocks':blocks, 'kernel_regular':kernel_reguler,
            'best_epoch':best_epoch, 'train_loss':train_loss, 
            'train_acc':train_acc, 'val_loss':val_loss, 'val_acc':val_acc})


In [15]:
df_res
    
    
    

,batch_size,epoch,learn_rate,learn_rate_decay,filters,drop,fc_size,blocks,kernel_regular,best_epoch,train_loss,train_acc,val_loss,val_acc
0,256,50,0.001,0.8,16,0.4,256,443,NaN,49,0.995155,0.011271,0.970613,0.218984
1,256,20,0.010,0.8,128,0.5,256,43,NaN,1,0.333209,10.740725,0.333756,10.738578
2,128,20,0.010,0.5,16,0.4,64,4441,NaN,20,0.797039,0.498250,0.817549,0.453982
3,256,40,0.001,0.8,16,0.2,64,43,NaN,40,0.981213,0.053976,0.966327,0.150773
4,512,50,0.010,0.1,128,0.4,32,441,NaN,1,0.333248,10.734761,0.333933,10.735728
5,512,40,0.100,0.8,128,0.5,64,4441,NaN,1,0.333170,10.735352,0.333933,10.735728
6,256,20,0.100,0.8,16,0.2,64,41,NaN,1,0.333160,10.742233,0.333933,10.735728
7,1024,50,0.100,0.8,128,0.4,256,41,NaN,1,0.333227,10.721587,0.333727,10.739059
8,1024,20,0.010,0.5,128,0.2,256,43,NaN,1,0.333505,10.716657,0.332371,10.760908
9,256,40,0.001,0.1,16,0.5,256,43,NaN,27,0.993998,0.013961,0.969874,0.193111
